# 导入基础数据包

In [ ]:
import ee
import geemap
import geemap.chart as chart
import numpy as np
import pandas as pd
import copy
import math
import time 
import traceback
import sys,os
os.chdir('/root/GEE')
# sys.path.append(os.path.dirname(os.path.abspath('__file__')))
from tqdm import trange,tqdm
from Basic_tools import *
from New_Correct import *
from Correct_filter import *
from S2_filter import *
from functools import partial
import geopandas as gpd
# from osgeo import gdal
# geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

# 函数汇总

## 函数：计算几何畸变区域，然后采用ImageCollection进行融合

In [28]:
def CalDitor(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],Origin_Method='RS',Method:str='1',
             AOI_buffer=None,Origin_scale=10,projScale=30): 
    
    volumetric_dict = my_slope_correction(s1_ascending,s1_descending,AOI_buffer,DEMNASA,Model,Origin_scale,DistorMethed=Origin_Method)

    # ----------------------Z-scor均衡化，直方图匹配，在局部进行直方图匹配
    Ascending_Img = volumetric_dict['ASCENDING'].select(Bands_)
    Descending_Img = volumetric_dict['DESCENDING'].select(Bands_)
    A_median = Ascending_Img.reduceRegion(**{
                          'reducer': ee.Reducer.median(),
                          'geometry':AOI_buffer
                        }).toArray()
    D_median = Descending_Img.reduceRegion(**{
                          'reducer': ee.Reducer.median(),
                          'geometry':AOI_buffer
                        }).toArray()
    Ascending_Img = Ascending_Img.subtract(A_median.subtract(D_median).getInfo())
    volumetric_dict['ASCENDING'] = replaceBands(volumetric_dict['ASCENDING'],Ascending_Img)


    # -----------------------基于线性关系。检测几何畸变
    Templist_A = AuxiliaryLine2Point(volumetric_dict['ASCENDING'],volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                                volumetric_dict['ASCENDING_parms']['Auxiliarylines'],
                                projScale)

    Templist_D = AuxiliaryLine2Point(volumetric_dict['DESCENDING'],volumetric_dict['DESCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['DESCENDING_parms']['coordinates_dict'],
                                volumetric_dict['DESCENDING_parms']['Auxiliarylines'],
                                projScale)
    if   Method == '1':
        LeftLayoverA,RightLayoverA,ShadowA = Line_Correct(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale,
                                                filt_distance=False, save_peak=False, line_points_connect=True,Peak_Llay=True,Peak_shdow=True,Peak_Rlay=True)
        LeftLayoverD,RightLayoverD,ShadowD =  Line_Correct(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale,
                                                filt_distance=False,save_peak=False,line_points_connect=True,Peak_Llay=True,Peak_shdow=True,Peak_Rlay=True)
    elif Method == '2':
        LeftLayoverA,RightLayoverA,ShadowA = Line_Correct2(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale)
        LeftLayoverD,RightLayoverD,ShadowD =  Line_Correct2(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale)
    else:
        print('Method = {}'.format(Method))
        raise Exception('Method方法目前只有两种，请重新输入')
    # 如果不加这一段，constant会判断出错
    LeftLayoverA=LeftLayoverA.rename('LeftLayoverA');RightLayoverA=RightLayoverA.rename('RightLayoverA');ShadowA=ShadowA.rename('ShadowA')
    LeftLayoverD=LeftLayoverD.rename('LeftLayoverD');RightLayoverD=RightLayoverD.rename('RightLayoverD');ShadowD=ShadowD.rename('ShadowD')
    return LeftLayoverA,RightLayoverA,ShadowA,Templist_A,LeftLayoverD,RightLayoverD,ShadowD,Templist_D,volumetric_dict,synthesis

def Combin_AB(LeftLayoverA,RightLayoverA,ShadowA,LeftLayoverD,RightLayoverD,ShadowD,
              volumetric_dict,AOI_buffer=None,restrict=False):   
    
    def Cal_mask(LeftLayover,RightLayover,Shadow,AOI_buffer):
        # 判断是否为空
        left_empty = LeftLayover.bandNames().length().eq(0)
        right_empty = RightLayover.bandNames().length().eq(0)
        shadow_empty = Shadow.bandNames().length().eq(0)

        # 只合并非空图像
        result = ee.Image(ee.Algorithms.If(left_empty, ee.Image(), LeftLayover))
        result = ee.Image(ee.Algorithms.If(right_empty,result,result.Or(RightLayover)))
        result = ee.Image(ee.Algorithms.If(shadow_empty,result,result.Or(Shadow)))
        return result.clip(AOI_buffer)

    LeftLayoverA_,RightLayoverA_,ShadowA_ = LeftLayoverA.mask().clip(AOI_buffer),RightLayoverA.mask().clip(AOI_buffer),ShadowA.mask().clip(AOI_buffer)
    LeftLayoverD_,RightLayoverD_,ShadowD_ = LeftLayoverD.mask().clip(AOI_buffer),RightLayoverD.mask().clip(AOI_buffer),ShadowD.mask().clip(AOI_buffer)

    A_mask_ = Cal_mask(LeftLayoverA_,RightLayoverA_,ShadowA_,AOI_buffer)
    D_mask_ = Cal_mask(LeftLayoverD_,RightLayoverD_,ShadowD_,AOI_buffer)
    All_distor = A_mask_.And(D_mask_)

    DistorDict = {'LeftLayoverA':LeftLayoverA,'RightLayoverA':RightLayoverA,'ShadowA':ShadowA, 'A_mask_':A_mask_,
                  'LeftLayoverD':LeftLayoverD,'RightLayoverD':RightLayoverD,'ShadowD':ShadowD,'D_mask_':D_mask_,
                  'All_distor'  :All_distor}
    
    # 判断是否存在畸变
    A_empty = A_mask_.bandNames().contains('constant')
    A_empty = ee.Number(ee.Algorithms.If(A_empty, 1, 0))
    D_empty = D_mask_.bandNames().contains('constant')
    D_empty = ee.Number(ee.Algorithms.If(D_empty, 1, 0))

    s1_ascending_flat  = volumetric_dict['ASCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])
    s1_descending_flat = volumetric_dict['DESCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])
    
    if restrict:
        # 严格。如果A、D都存在畸变，则该区域为空
        s1_ascending_ =  ee.Image(ee.Algorithms.If(A_empty, s1_ascending_flat,s1_ascending_flat.updateMask(A_mask_.Not())))
        s1_descending_=  ee.Image(ee.Algorithms.If(D_empty,s1_descending_flat,s1_descending_flat.updateMask(D_mask_.Not())))                            
        Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])
        s1_unit_mean_ = Combin_AD.mean().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
        s1_unit_max_ = Combin_AD.max().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
        s1_unit_min_ = Combin_AD.min().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
    else:
        # 宽松，如果存在畸变，则优先采用逆轨影像替换
        s1_ascending_ = ee.Image(ee.Algorithms.If(A_empty, ee.Image(), s1_ascending_flat.where(A_mask_,s1_descending_flat)))
        s1_descending_ = ee.Image(ee.Algorithms.If(D_empty, ee.Image(), s1_descending_flat.where(D_mask_,s1_ascending_flat)))
        Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])
        s1_unit_mean_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),s1_ascending_flat.add(s1_descending_flat).divide(2),Combin_AD.mean())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

        s1_unit_max_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                    s1_ascending_flat.max(s1_descending_flat),
                    Combin_AD.max())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

        s1_unit_min_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                    s1_ascending_flat.min(s1_descending_flat),
                    Combin_AD.min())).reproject(volumetric_dict['ASCENDING_parms']['proj'])
    
    return s1_unit_mean_,s1_unit_max_,s1_unit_min_,DistorDict

## 函数：提取冰湖，检测效果

### 聚类方法

In [ ]:
from Extract_algorithm import Cluster_extract as Cluster

def Cluster_math(method: str, img, bands: list, index: str, visual: bool, save: bool, region=None,proj=None):
    '''method ('Kmean','SNIC','SNIC_Kmean','LVQ','Xmeans','Cobweb','CascadeKMeans')'''
    img = img.select(bands).clip(region)
    img = minmax_norm(img,bands,region,scale=Origin_scale,withbound=False)

    if proj:
        img = img.reproject(proj)
    if method == 'Kmean':
        result = Cluster.afn_Kmeans(img, region)
    elif method == 'Cobweb':
        result = Cluster.afn_Cobweb(img, region)
    elif method == 'Xmeans':
        result = Cluster.afn_Xmeans(img, region)
    elif method == 'LVQ':
        result = Cluster.afn_LVQ(img, region)
    elif method == 'CascadeKMeans':
        result = Cluster.afn_CascadeKMeans(img, region)
    elif method == 'SNIC':
        result = Cluster.afn_SNIC(img)
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(proj)
    elif method == 'SNIC_Kmean':
        result = Cluster.afn_SNIC(img)
        # 默认舍弃cluster和seed
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(proj)
        result0 = Cluster.afn_Kmeans(result, region)  # 原始图像不参与
        result1 = Cluster.afn_Kmeans(result.addBands(img), region)  # 原始图像参与    .unmask(10)

    if visual:
        Map = geemap.Map(basemap='HYBRID')  #
        Map.centerObject(region, zoom=15)
        Map.addLayer(img.select(0), {'min': 0, 'max': 1}, 'Origin')
        if method in ['Kmean', 'Cobweb', 'Xmeans', 'LVQ', 'CascadeKMeans']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC':
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC_Kmean':
            Map.addLayer(result0.randomVisualizer(), {}, 'SNIC_Kmean_NoOrigin')
            Map.addLayer(result1.randomVisualizer(), {}, 'SNIC_Kmean_YesOrigin')
        else:
            print('Please check your method str')
    else:
        Map = None
    if save:
        if method == 'SNIC_Kmean':
            Geemap_export(filename=index + 'NoOrigin' + method + '.tif', collection=False, image=result0,
                          region=region, scale=10)
            Geemap_export(filename=index + 'YesOrigin' + method + '.tif', collection=False, image=result1,
                          region=region, scale=10)
        else:
            Geemap_export(filename=index + method + '.tif', collection=False, image=result, region=region, scale=10)
        pass
    if method == 'SNIC_Kmean':
        return Map, result0.addBands(result1)
    else:
        return Map, result

### 自适应阈值

In [ ]:
from Extract_algorithm import Adaptive_threshold as Adap

def Bandmath(method:str,img,band,index:str,visual:bool,save:bool,region=None,proj=None):
    '''
    method = ['otsu','minimum','yen','isodata']
    img: 仅单波段图像
    '''
    img = img.select(band).clip(region)
    img = minmax_norm(img,band,region,scale=Origin_scale,withbound=False)
    img = img.reproject(proj).clip(region)

    if method == 'otsu':
        histogram = get_histogram(img,region=region,scale=Origin_scale)
        Threshould_value = Adap.afn_otsu(histogram)
        result = img.select(0).gt(Threshould_value).clip(region)  #
        print('Threshould value is {}'.format(Threshould_value.getInfo()))

    elif method == 'minimum':
        bin_centers, counts,_  = GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_minimum(bin_centers, counts,max_num_iter = 10000)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))
        
    elif method == 'yen':
        bin_centers, counts,_ = GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_yen(bin_centers, counts)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))
    
    elif method == 'isodata':
        bin_centers, counts, bucketWidth = GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_isodata(bin_centers, counts,bucketWidth)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))

    if visual:
        Map = geemap.Map(basemap='HYBRID') #
        Map.centerObject(region, zoom=15)
        Map.addLayer(img.select(0), {'min': 0, 'max': 1}, 'Origin')
        if method in ['otsu','minimum','yen','isodata']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        else:
            print('Wrong visual! Please check your method str')
    else:
        Map = None

    if save:
        if method in ['otsu','minimum','yen','isodata']:
            Geemap_export(filename=index+method+'.tif',collection=False,
                          image=result,region=region,scale=10)
        else:
            print('Wrong save! Please check your method str')

    return Map,result

# 数据加载,与参数设置

In [ ]:
#--------------------------预加载冰湖数据
Glacial_lake = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/SAR_GLs/2019Gls_SARExt')

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_C = Glacial_lake.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_R = Glacial_lake.map(Rectangle_set)             # 计算最小包络矩形

#抽取属性作为list
# Glacial_lake_A_GeoList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Num_list = Glacial_lake.size().getInfo()
Glacial_lake_A_GeoList = Glacial_lake.toList(Num_list)
Glacial_lake_C_CentriodList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_R_RectangleList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))


#--------------------确定时间范围
# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
year = '2019'

START_DATE  = ee.Date(year + '-06-01')
END_DATE   = ee.Date(year + '-08-30')
S2_START_DATE = ee.Date(year + '-06-01')
S2_END_DATE  = ee.Date(year + '-09-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')

#-------------------DEM选择
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)

#-------------------几何畸变参数
models = ['volume', 'surface', None]     # 地形矫正模型
Model = models[0]
Origin_scale = 10
projScale = 30

#-------------------S2 clodless参数
CLOUD_FILTER = 60         # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15        # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15       # 非水暗像素判断阈值
CLD_PRJ_DIST = 1         # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50            # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input

# ------------------分类方法和参数确定
from Extract_algorithm import Reprocess,save_parms
save_dir='/root/GEE/{}补齐结果'.format(year)
os.chdir(save_dir)
Methods = ('Kmean','SNIC_Kmean','Xmeans','CascadeKMeans','otsu','isodata')
resultbands = (0,1)
Imgs = ('s1_unit_mean_',) # 's1_unit_max_','s1_unit_min_'
Bands = (['VV_gamma0_flatDB'],['VV_gamma0_flatDB','VH_gamma0_flatDB'])
mode = 'gpd'
savenmae = '{}SouthwestGLs'.format(year)

# ------------------重载变量参数
Reload = True
dest = 'users/mrwurenzhe/test'
metaName = 'test.csv'
eeUser = 'mrwurenzhe@furwas.com'
overwrite = 'yes'
delgeeUp = False    # 删除gee远端存储

# 输出图像

## 输出

In [ ]:
for i in trange(4700,Num_list):
    IoU_All = [] ; Wrong_dataIndex = []
    try:
        start_time = time.time()
        AOI_Centriod = ee.Feature.geometry(Glacial_lake_C_CentriodList.get(i))
        AOI_GLFeature = ee.Feature(Glacial_lake_A_GeoList.get(i))
        AOI_GL = AOI_GLFeature.geometry()
        AOI_Rectangle = ee.Feature.geometry(Glacial_lake_R_RectangleList.get(i))

        # 计算面积
        # AOI_area = AOI_GL.area().divide(ee.Number(1000*1000)).getInfo()
        # AOI_perometer = AOI_GL.perimeter().divide(1000).getInfo()
        # circularity = AOI_area*3.1415926*4/AOI_perometer**2
        AOI_area = AOI_GLFeature.get('GL_Area_km').getInfo()
        circularity = AOI_GLFeature.get('GL_Circula').getInfo()

        # 缩小冰湖面积
        AOI_ShrinKbuffer = AOI_GL.buffer(distance=math.log(AOI_area+1,10) * -3000 *circularity)
        AOI_ShrinKbuffer_Centroid = AOI_ShrinKbuffer.centroid()

        # 检测是否内缩为空，否则用AOI_area替代
        if AOI_ShrinKbuffer.coordinates().getInfo() != [] :
            FilterBound = AOI_ShrinKbuffer_Centroid
        else:
            FilterBound = AOI_Centriod

        # 边界不可过大，否则会影响局部自适应提取效果
        AOI_buffer = AOI_Rectangle.buffer(distance=math.log(AOI_area+1,5) * 1200 + 100).bounds()

        # 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
        s1_ascending, s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,Filter='RefinedLee',FilterSize=30)

        # 采用S2cloudless生产一张合成的无云图像
        s2_sr_median = merge_s2_collection(AOI_buffer,START_DATE,END_DATE,CLOUD_FILTER,BUFFER,CLD_PRJ_DIST,CLD_PRB_THRESH,NIR_DRK_THRESH)
        
        # 判断是否synth
        synthesis_a  = ee.Image(s1_ascending).get('synthesis').getInfo()
        synthesis_d  = ee.Image(s1_descending).get('synthesis').getInfo()
        if synthesis_a or synthesis_d:synthesis=1        
        else:synthesis=0

        if synthesis:
            print('Image with synthesis')
            s1_ascending = s1_ascending.rename(['VV_gamma0_flatDB','VH_gamma0_flatDB','incAngle'])
            s1_descending = s1_descending.rename(['VV_gamma0_flatDB','VH_gamma0_flatDB','incAngle'])
            default_prj = s1_ascending.select(0).projection()
            Combin_ad = ee.ImageCollection([s1_ascending,s1_descending])
            s1_unit_mean_ = Combin_ad.mean().reproject(default_prj).clip(AOI_buffer)
            s1_unit_max_  = Combin_ad.max().reproject(default_prj).clip(AOI_buffer)
            s1_unit_min_  = Combin_ad.min().reproject(default_prj).clip(AOI_buffer)
            volumetric_dict = {} 
            volumetric_dict['ASCENDING'] = s1_ascending.clip(AOI_buffer)
            volumetric_dict['ASCENDING_parms'] = {'proj':default_prj}
            volumetric_dict['DESCENDING'] = s1_descending.clip(AOI_buffer)
            volumetric_dict['DESCENDING_parms'] = {'proj':s1_descending.select(0).projection()}
        else:
            # 几何畸变检测与升降轨图像融合
            LeftLayoverA,RightLayoverA,ShadowA,Templist_A,LeftLayoverD,RightLayoverD,ShadowD,Templist_D,volumetric_dict,synthesis = \
                    CalDitor(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],Origin_Method='RS',Method='1',
                            AOI_buffer=AOI_buffer,Origin_scale=Origin_scale,projScale=projScale)
            
            s1_unit_mean_,s1_unit_max_,s1_unit_min_,DistorDict = \
                    Combin_AB(LeftLayoverA,RightLayoverA,ShadowA,LeftLayoverD,RightLayoverD,ShadowD,volumetric_dict,AOI_buffer=AOI_buffer)
       
        if synthesis_a == 0:
            a_name,a_date,a_nodata = ee.List([s1_ascending.get('system:index'),s1_ascending.date().format('YYYY-MM-dd'),s1_ascending.get('numNodata')]).getInfo()
        else:
            a_name = 'None'; a_date = 'None' ; a_nodata='All_have'

        if synthesis_d == 0:
            d_name,d_date,d_nodata = ee.List([s1_descending.get('system:index'),s1_descending.date().format('YYYY-MM-dd'),s1_descending.get('numNodata')]).getInfo()
        else:
            d_name = 'None'; d_date = 'None' ; d_nodata='All_have'

        pd_dict = {'a_name':a_name,'d_name':d_name,'a_date':a_date,
                   'd_date':d_date,'a_nodata':a_nodata,'d_nodata':d_nodata}

        K = [1 if Method == 'SNIC_Kmean' else 0 for Method in Methods ]


        for k,Method in zip(K,Methods):
            for img in Imgs:
                for Band in Bands:
                    Bands = (['VV_gamma0_flatDB'],['VV_gamma0_flatDB','VH_gamma0_flatDB'])

                    if Method in ['Kmean','SNIC_Kmean','LVQ','Xmeans','CascadeKMeans']:
                        if len(Band)==2:
                            Map,result = Cluster_math(method=Method,img=eval(img),bands=Band,index='',
                                                        visual=False,save=False,region=AOI_buffer,
                                                        proj=volumetric_dict['ASCENDING_parms']['proj'])
                        else:
                            print('Method 1 {} Band={},continue'.format(Method,Band))
                            continue

                    elif Method in ['otsu','minimum','yen','isodata']:
                        if len(Band)==1:
                            Map,result = Bandmath(method=Method,img=eval(img),band=Band,index='',
                                    visual=False,save=False,region=AOI_buffer,
                                    proj=volumetric_dict['ASCENDING_parms']['proj'])
                        else:
                            print('Method 2 {} Band={},continue'.format(Method,Band))
                            continue
                    else:
                        print('Wrong method input!')

                    for resultband in range(k+1):

                        # 分类图转矢量,FilterBound可以选择AOI_point或者FilterBuffer
                        Union_ex = Reprocess.image2vector(result,resultband=resultband,GLarea=AOI_area,FilterBound=FilterBound,del_maxcount=False)
                        logname = savenmae+f'{i//100 * 100}_{i//100*100+100}.csv'
                        shpname = savenmae+f'{i//100 * 100}_{i//100*100+100}.shp'
                        # 导出csv和shp
                        Result_dict = save_parms.write_pd(Union_ex,i,AOI_GL,img,mode=mode,Method=Method,Band=Band,WithOrigin=resultband,pd_dict=pd_dict,
                                              Area_real=AOI_area,logname=logname,shapname=shpname,calIoU=True,cal_resultArea=False,returnParms=True)
                        IoU_All.append(Result_dict['IoU'])
                        print('Write csv and shp, Method={}  Img={}  Band={}'.format(Method,img,Band))

        # 当IoU比较小的时候，IoU<0.6下载图像
        if np.sum(np.array(IoU_All)>0.6) == 0:
                Geemap_export(fileDirname=f'{i:04d}'+'_'+'Ascending_'+a_name+'.tif',collection=False,image=volumetric_dict['ASCENDING'],region=AOI_buffer,scale=10)
                Geemap_export(fileDirname=f'{i:04d}'+'_'+'Descending_'+d_name+'.tif',collection=False,image=volumetric_dict['DESCENDING'],region=AOI_buffer,scale=10)
                Geemap_export(fileDirname=f'{i:04d}'+'_'+'s2a_sr_median'+'.tif',collection=False,image=s2_sr_median,region=AOI_buffer,scale=10)   
            
                # 导出融合后的结果图像
                Geemap_export(fileDirname=f'{i:04d}'+'_'+'s1_unit_mean_'+'.tif',collection=False,image=s1_unit_mean_,region=AOI_buffer,scale=10)
                Geemap_export(fileDirname=f'{i:04d}'+'_'+'s1_unit_max_'+'.tif',collection=False,image=s1_unit_max_,region=AOI_buffer,scale=10)
                Geemap_export(fileDirname=f'{i:04d}'+'_'+'s1_unit_min_'+'.tif',collection=False,image=s1_unit_min_,region=AOI_buffer,scale=10)
                
                
                if synthesis!=1:
                    # 导出几何畸变图像
                    DistorA = LeftLayoverA.rename('LeftLayoverA').addBands(RightLayoverA.rename('RightLayoverA')).addBands(ShadowA.rename('ShadowA'))
                    if DistorA.bandNames().length().getInfo() != 0:
                        Geemap_export(fileDirname=f'{i:04d}'+'_'+'DistortionA'+a_name+'.tif',image=DistorA,region=AOI_buffer,scale=10)

                    DistorD = LeftLayoverD.rename('LeftLayoverD').addBands(RightLayoverD.rename('RightLayoverD')).addBands(ShadowD.rename('ShadowD'))
                    if DistorD.bandNames().length().getInfo() != 0:
                        Geemap_export(fileDirname=f'{i:04d}'+'_'+'DistortionD'+d_name+'.tif',image=DistorD,region=AOI_buffer,scale=10) 

        end_time = time.time() 
        print('该函数运行了 %.5f 秒' % (end_time - start_time))
    except:
        # 记录错误信息
        Wrong_dataIndex.append(i)
        with open('log.txt', 'a') as f: 
            f.write('Wrong index = {}\n'.format(1))
            f.write(traceback.format_exc())
            f.write('\n')
        print('错误已记录到log.txt')

  0%|          | 0/244 [00:00<?, ?it/s]

Begin Filter ...



100%|##########| 15/15 [00:00<00:00, 1819.08it/s]

100%|##########| 15/15 [00:00<00:00, 361.34it/s]

100%|##########| 15/15 [00:00<00:00, 1375.99it/s]

100%|##########| 15/15 [00:00<00:00, 754.97it/s]


Method 1 Kmean Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 SNIC_Kmean Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=SNIC_Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Write csv and shp, Method=SNIC_Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 Xmeans Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=Xmeans  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 CascadeKMeans Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=CascadeKMeans  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Threshould value is 0.4754641512130742
Write csv and shp, Method=otsu  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB']
Method 2 otsu Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'],continue


/root/GEE/Extract_algorithm.py:229: RuntimeWarning: invalid value encountered in divide
  higher = (csum_intensity[-1] - csum_intensity[:-1]) / (csumh[:-1]+sys.float_info.min)


...

Method 1 Kmean Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 SNIC_Kmean Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=SNIC_Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Write csv and shp, Method=SNIC_Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 Xmeans Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=Xmeans  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 CascadeKMeans Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=CascadeKMeans  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Threshould value is 0.4660707399998214
Write csv and shp, Method=otsu  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB']
Method 2 otsu Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'],continue
Threshould value is 0.4677734375
Write csv and shp, Method=isodata  Img=s1_unit_mean_  Band=

 85%|########4 | 207/244 [9:36:42<1:34:19, 152.97s/it]

Begin Filter ...



100%|##########| 16/16 [00:00<00:00, 1886.51it/s]

100%|##########| 16/16 [00:00<00:00, 300.52it/s]

100%|##########| 18/18 [00:00<00:00, 1723.88it/s]

100%|##########| 18/18 [00:00<00:00, 366.14it/s]


Method 1 Kmean Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 SNIC_Kmean Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=SNIC_Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Write csv and shp, Method=SNIC_Kmean  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 Xmeans Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=Xmeans  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Method 1 CascadeKMeans Band=['VV_gamma0_flatDB'],continue
Write csv and shp, Method=CascadeKMeans  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB']
Threshould value is 0.5339752053316318
Write csv and shp, Method=otsu  Img=s1_unit_mean_  Band=['VV_gamma0_flatDB']
Method 2 otsu Band=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'],continue
Threshould value is 0.5166684520560687
Write csv and shp, Method=isodata  Img=s1_unit_mean_ 